In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from spacy.cli.train import train
import spacy
from translate import Translator
from concurrent.futures import ThreadPoolExecutor
import time
import translators as ts
import requests


Using region Tel Aviv server backend.



In [ ]:
df = pd.read_csv("Data/Predicted_DB.csv")
translated_texts = []
df = df.dropna(subset=['Text'])
df = df[df['Text'].apply(len) >= 40]


with open("translated_texts.txt", "w", encoding="utf-8") as f:
    for idx, text in enumerate(df['Text']):
        f.write(f"@idx@{text}\n")  
        
with open("translated_texts.txt","r") as f:
    text = f.read()

chunks = [text[i:i+1000] for i in range(0, len(text), 1000)]
print(len(chunks))

for chunk in chunks:
    print(len(translated_texts))
    try:
        translated_chunk = ts.translate_text(chunk)
        translated_texts.append(translated_chunk)
        time.sleep(60)
    except requests.exceptions.HTTPError as e:
        if e.response.status_code == 429:  # If it's a 'Too Many Requests' error
            print("Rate limit reached. Waiting for 60 seconds before retrying...")
            time.sleep(60)
            try:
                translated_chunk = ts.translate_text(chunk)
                translated_texts.append(translated_chunk)
                print("Work")
                time.sleep(60)
            except:
                time.sleep(60)

        else:
            print(f"An error occurred: {e}")

translated_texts
print(ts.translate_text(text[:1000]))



171
0
Rate limit reached. Waiting for 60 seconds before retrying...
Work
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
Rate limit reached. Waiting for 60 seconds before retrying...
26
27
28
29
30


In [ ]:
import re

def add_translated_texts_to_df(df, translated_texts):
    full_translated_text = ''.join(translated_texts)
    # Use regex to split lines starting with a number and a period
    lines = re.split(r'^\d+\.\s*', full_translated_text, flags=re.MULTILINE)

    # Create an empty list to store the translations
    translations = []

    for line in lines:
        if line.strip():  # Check if the line is not empty
            translations.append(line.strip())

    # Add translations to the dataframe
#     df['Text_en'] = translations

    return translations

df = add_translated_texts_to_df(df, translated_texts)

df
